In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
ostro_dir = 'Osteosarcoma-UT'



# Define image dimensions and batch size
img_height = 224
img_width = 224
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=20,  # Random rotation
    width_shift_range=0.2,  # Random horizontal shift
    height_shift_range=0.2,  # Random vertical shift
    shear_range=0.2,  # Shear transformation
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Random horizontal flip
    fill_mode='nearest'
)


test_datagen = ImageDataGenerator(
    rescale=1./255
)

In [3]:
from sklearn.model_selection import train_test_split
import os

# Define the root directory containing subfolders for each class
root_dir = 'Osteosarcoma-UT'

# Get the list of class names
classes = os.listdir(root_dir)
print(classes)

# Split the data into training and validation sets
train_data = []
val_data = []
for cls in classes:
    cls_dir = os.path.join(root_dir, cls)
    images = [os.path.join(cls_dir, img) for img in os.listdir(cls_dir)]
    train_images, val_images = train_test_split(images, test_size=0.2, random_state=42)
    train_data.extend([(img, cls) for img in train_images])
    val_data.extend([(img, cls) for img in val_images])

# Further split the training data into training and testing sets
train_images, test_images = train_test_split(train_data, test_size=0.2, random_state=42)

# Now you have train_images, val_data, and test_images containing paths to images for each set



['Viable', 'Non-Tumor', 'Non-Viable-Tumor']


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming you have a DataFrame with two columns: 'file_path' and 'label'
train_df = pd.DataFrame(train_data, columns=['file_path', 'label'])
val_df = pd.DataFrame(val_data, columns=['file_path', 'label'])
test_df = pd.DataFrame(test_images, columns=['file_path', 'label'])

# Split the data into file paths and labels
train_data = train_df['file_path']
train_labels = train_df['label']

val_data = val_df['file_path']
val_labels = val_df['label']

test_data = test_df['file_path']
test_labels = test_df['label']

# You can create separate data generators for each set
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='file_path',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

validation_generator = test_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='file_path',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='file_path',
    y_col='label',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


Found 914 validated image filenames belonging to 3 classes.


Found 230 validated image filenames belonging to 3 classes.
Found 183 validated image filenames belonging to 3 classes.


In [5]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [6]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Dropout

img_height =224
img_width = 224

# Load the pre-trained DenseNet121 model without the top (fully connected) layers
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Add your own classification layers on top of the pre-trained model
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global average pooling layer
x = Dense(512, activation='relu')(x)  # Fully connected layer
x = Dropout(0.2)(x)  # Dropout layer for regularization
predictions = Dense(3, activation='softmax')(x)  # Output layer

# Define the model to be trained
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model with categorical_crossentropy as the loss function
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model
history = model.fit(train_generator,
                    steps_per_epoch=len(train_generator),
                    epochs=100,
                    validation_data=validation_generator,
                    validation_steps=len(validation_generator),
                    callbacks=[early_stopping])


2024-02-22 10:29:04.261566: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-02-22 10:29:04.261589: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-02-22 10:29:04.261601: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-02-22 10:29:04.261902: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-22 10:29:04.262230: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 230, 230, 3)          0         ['input_1[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv1/conv (Conv2D)         (None, 112, 112, 64)         9408      ['zero_padding2d[0][0]']      
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1/conv[0][0]']      

2024-02-22 10:29:09.706115: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


29/29 [==============================] - 28s 816ms/step - loss: 1.0156 - accuracy: 0.6663 - val_loss: 0.7563 - val_accuracy: 0.7522
Epoch 2/100
29/29 [==============================] - 18s 606ms/step - loss: 0.4537 - accuracy: 0.8392 - val_loss: 0.6674 - val_accuracy: 0.7696
Epoch 3/100
29/29 [==============================] - 18s 627ms/step - loss: 0.4550 - accuracy: 0.8370 - val_loss: 0.6762 - val_accuracy: 0.7696
Epoch 4/100
29/29 [==============================] - 20s 675ms/step - loss: 0.3927 - accuracy: 0.8643 - val_loss: 0.6552 - val_accuracy: 0.7783
Epoch 5/100
29/29 [==============================] - 19s 663ms/step - loss: 0.4010 - accuracy: 0.8534 - val_loss: 0.4659 - val_accuracy: 0.8522
Epoch 6/100
29/29 [==============================] - 18s 614ms/step - loss: 0.3413 - accuracy: 0.8753 - val_loss: 0.5350 - val_accuracy: 0.8130
Epoch 7/100
29/29 [==============================] - 19s 652ms/step - loss: 0.3805 - accuracy: 0.8764 - val_loss: 0.4562 - val_accuracy: 0.8609
Epoc

In [7]:
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print("Test accuracy:", test_acc)

5/5 [==============================] - 2s 396ms/step - loss: 0.2405 - accuracy: 0.9125
Test accuracy: 0.9125000238418579


In [9]:
from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score
import numpy as np

# Get predictions and true labels
# Ensure that steps is set to get the correct number of samples
y_pred = model.predict_generator(test_generator, steps=test_generator.samples // batch_size + 1)
y_true = test_generator.classes

# Only take the necessary number of samples from y_true
y_true = y_true[:len(y_pred)]

# Calculate macro-averaged F1-score
macro_f1 = f1_score(y_true, y_pred.argmax(axis=1), average='macro')
print("Macro-averaged F1-score:", macro_f1)

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred.argmax(axis=1))
print("Confusion matrix:\n", cm)

# Calculate AUC-ROC score for each class
y_onehot = np.zeros((len(y_true), len(np.unique(y_true))))
y_onehot[np.arange(len(y_true)), y_true] = 1
auc_roc_scores = roc_auc_score(y_onehot, y_pred, multi_class='ovr')
print("AUC-ROC scores for each class:", auc_roc_scores)

# Calculate average AUC-ROC score
average_auc_roc = np.mean(auc_roc_scores)
print("Average AUC-ROC score:", average_auc_roc)
2

/var/folders/yd/6l3qbbqj2qx_y_lxrr45xd480000gn/T/ipykernel_18908/3247351702.py:6: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  y_pred = model.predict_generator(test_generator, steps=test_generator.samples // batch_size + 1)


Macro-averaged F1-score: 0.9043290888162218
Confusion matrix:
 [[87  4  2]
 [ 3 39  2]
 [ 3  2 41]]
AUC-ROC scores for each class: 0.9833079313066572
Average AUC-ROC score: 0.9833079313066572


2